# Sommaire

>[Sommaire](#scrollTo=OXXSgYy9BBec)

>[Imports et paramètres généraux](#scrollTo=zC7YhIrj59gy)

>[Création des modèles à entraîner](#scrollTo=gZFVwEW96EaG)

>[Fonctions d'affichages et d'enregistrements](#scrollTo=m_Wx43PK6pdE)

>[Entraînement](#scrollTo=5CAxdPTM6wkq)

>[Téléchargements](#scrollTo=4QJiqfKLBU_W)



# Imports et paramètres généraux

Imports des librairies principales et réglage des paramètres d'aléa

In [18]:
!pip install tqdm

# gestions relatives aux fichiers de manière générale
import os

# choix de tensorflow comme backend de Keras
os.environ["KERAS_BACKEND"] = "tensorflow"

# opérations matricielles
import numpy as np

from tqdm import tqdm

# affichages
import matplotlib.pyplot as plt

# les différentes parties utilisées de Keras
from keras.layers import Input
from keras.models import Model, Sequential
from keras.layers.core import Reshape, Dense, Dropout, Flatten
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import Convolution2D, UpSampling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras import backend as K
from keras import initializers

K.set_image_dim_ordering('th')

# on fixe le seed du random de numpy pour avoir des résultats reproductibles
np.random.seed(1000)

# On choisit la taille du vecteur aléatoire d'entrée du GAN
# 10 marche bien pour MNIST d'après la littérature
randomDim = 10

Chargement du dataset

In [0]:
# Chargement directement depuis Keras
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Normalisation des données
X_train = (X_train.astype(np.float32) - 127.5)/127.5

# Mise en forme des données d'entraînement
X_train = X_train.reshape(60000, 784)

# Création des modèles à entraîner

Configuration du Générateur

In [0]:
# Architecture trouvée dans la littérature, marche bien pour MNIST
generator = Sequential()
generator.add(Dense(256, input_dim=randomDim, kernel_initializer=initializers.RandomNormal(stddev=0.02)))
generator.add(LeakyReLU(0.2))
generator.add(Dense(512))
generator.add(LeakyReLU(0.2))
generator.add(Dense(1024))
generator.add(LeakyReLU(0.2))
generator.add(Dense(784, activation='tanh'))
generator.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5))

Configuration du Discriminateur

In [0]:
# Architecture trouvée dans la littérature, marche bien pour MNIST
discriminator = Sequential()
discriminator.add(Dense(1024, input_dim=784, kernel_initializer=initializers.RandomNormal(stddev=0.02)))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Dropout(0.3))
discriminator.add(Dense(512))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Dropout(0.3))
discriminator.add(Dense(256))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Dropout(0.3))
discriminator.add(Dense(1, activation='sigmoid'))
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5))

Configuration du GAN comme assemblage du Générateur et du Discriminateur

In [0]:
discriminator.trainable = False
ganInput = Input(shape=(randomDim,))
x = generator(ganInput)
ganOutput = discriminator(x)
gan = Model(inputs=ganInput, outputs=ganOutput)
gan.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5))

# Fonctions d'affichage et d'enregistrement

In [0]:
# Affichage de la fonction de coût
def plotLoss(epoch):
    plt.figure(figsize=(10, 8))
    plt.plot(dLosses, label='Discriminitive loss')
    plt.plot(gLosses, label='Generative loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig('gan_loss_epoch_%d.png' % epoch)

# Affichage d'une mosaïque d'images générées par le Générateur
def plotGeneratedImages(epoch, examples=100, dim=(10, 10), figsize=(10, 10)):
    noise = np.random.normal(0, 1, size=[examples, randomDim])
    generatedImages = generator.predict(noise)
    generatedImages = generatedImages.reshape(examples, 28, 28)

    plt.figure(figsize=figsize)
    for i in range(generatedImages.shape[0]):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(generatedImages[i], interpolation='nearest', cmap='gray_r')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig('gan_generated_image_epoch_%d.png' % epoch)

In [0]:
# sauvegarde des réseaux de neurones
def saveModels(epoch):
    generator.save('gan_generator_epoch_%d.h5' % epoch)
    discriminator.save('gan_discriminator_epoch_%d.h5' % epoch)

# **Entraînement avec mesure de complexité**


Il nous faut deux fonctions : une qui entraîne un GAN et renvoie une matrice des brillances pour chaque exemple à chaque epoch, et une qui traite cette matrice pour déterminer la complexité de chaque exemple.
### REMARQUE IMPORTANTE : train_on_batch peut pondérer les classes testées. Cf https://keras.io/models/sequential/

In [0]:
def trainWithComplexitySave(epochs=1, batchSize=1, data=0, nbEchantillons = 0):
    
    
    complexityMatrix = list()
    
    for i in range(epochs) :
      complexityMatrix.append([])
        
    
    batchCount = X_train.shape[0] / batchSize
    
    # Affichage des paramètres de train
    print 'Epochs:', epochs
    print 'Batch size:', batchSize
    print 'Batches per epoch:', batchCount

    # ENTRAINEMENT
    
    
    # Pour chaque epoch
    for e in xrange(1, epochs+1):
      
        # Affichage du numéro de l'epoch
        print '-'*15, 'Epoch %d' % e, '-'*15
        
        numEchantillon = 0
        
        # Pour chaque batch
        for j in tqdm(xrange(batchCount)):
            # On génère des données random de la bonne taille pour générer <batchSize> fausses images 
            noise = np.random.normal(0, 1, size=[batchSize, randomDim])
            imageBatch = X_train[np.random.randint(0, X_train.shape[0], size=batchSize)]

            # On génère des fausses images à partir du bruit généré
            generatedImages = generator.predict(noise)
            X = np.concatenate([imageBatch, generatedImages])

            
            yDis = np.zeros(2*batchSize)
            yDis[:batchSize] = 0.9
            yDis[batchSize:] = 0.1

            # Entrainement du Discriminateur
            discriminator.trainable = True
            dloss = discriminator.train_on_batch(X, yDis)
            
###################################################################################################################
            #Enregistrement de la brillance du neurone sur une donnée
            discriminator.trainable = False
            complexityMatrix[e-1].append(discriminator.evaluate(imageBatch,yDis[:batchSize], batch_size = 1))
###################################################################################################################

            # Entrainement du Générateur, le Discriminateur étant immuable
            noise = np.random.normal(0, 1, size=[batchSize, randomDim])
            yGen = np.ones(batchSize)
            discriminator.trainable = False
            gloss = gan.train_on_batch(noise, yGen)

        # On enregistre les données relatives à l'entrainement régulièrement
        if e == 1 or e % 20 == 0:
            plotGeneratedImages(e)
            saveModels(e)
            
    
    print(complexityMatrix)
    return(complexityMatrix)

In [0]:
iterations = 30
batchSize = 1

# C'est parti !

  

In [0]:
def matrixToComplexity(matrix):
  nbEpoch = len(matrix)
  nbExemple = len(matrix[0])
  seuil = 0.6  #A ajuster
 #seuil = np.median(np.array(matrix))   #On utilise la médiane pour définir la brillance seuil qui sert à déterminer la complexité
  proportionSeuil = 0.75   #A ajuster en fonction des résultats
  
  result=[0 for i in range (nbExemple)]
  
  for i in range (nbExemple):
    for j in range (nbEpoch) :
      nbSousSeuil = 0
      nbSurSeuil = 0
      test = matrix[j:]
      for k in range (len(test)):
        if test[k]>= seuil :
          nbSurSeuil +=1
        else :
          nbSousSeuil +=1
      if (nbSousSeuil/nbSurSeuil)>proportionSeuil :
        complexity = j
      break
    result[i]=complexity
  return (result)    #Liste de taille (nbExemples) contenant la complexité de chacun

#Plus la mesure de complexité est faible, plus la donnée est complexe. 
        

  
print(matrixToComplexity(trainWithComplexitySave(iterations, batchSize)))


  0%|          | 0/60000 [00:00<?, ?it/s]

Epochs: 30
Batch size: 1
Batches per epoch: 60000
--------------- Epoch 1 ---------------
1/1 [==============================] - 0s 96ms/step



  0%|          | 1/60000 [00:02<39:42:58,  2.38s/it]

1/1 [==============================] - 0s 2ms/step



  0%|          | 4/60000 [00:02<10:18:12,  1.62it/s]

...

1/1 [==============================] - 0s 2ms/step



  8%|▊         | 4788/60000 [02:21<27:15, 33.75it/s]

1/1 [==============================] - 0s 2ms/step



  8%|▊         | 4792/60000 [02:21<27:15, 33.75it/s]

1/1 [==============================] - 0s 2ms/step


# Téléchargements

In [0]:
from google.colab import files

for e in xrange(1, 300+1):
    if e == 1 or e % 20 == 0:
        files.download('gan_generated_image_epoch_%d.png' % e)
        files.download('gan_generator_epoch_%d.h5' % e)
        files.download('gan_discriminator_epoch_%d.h5' % e)

MessageError: ignored